In [1]:
import utils as Utils
from pytorch_dqn_agent import DQN_Agent_PyTorch
from agent_handler import Agent_handler
from assignment3_utils import *
import numpy as np
import gym

In [2]:
env = gym.make('PongDeterministic-v4', render_mode='rgb_array')

num_obs, num_actions = Utils.describe_env(env)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Observation space:  Box(0, 255, (210, 160, 3), uint8)
Observation space size:  (210, 160, 3)
Number of actions:  6


In [3]:
def get_params(batch, update):
  obs = (4, 81, 70)
  params = {
    "num_obs": obs,
    "num_actions": num_actions,
    "update_rate": update,
    "learning_rate": 0.0001,
    "discount_factor": 0.95,
    "exploration_factor": 1,
    "min_exploration_rate": 0.05,
    "exploration_decay": 0.995,
    "batch_size": batch,
    "name": f'{batch}_{update}'
  }
  return params

In [11]:
agents = []

batchs = [8, 16]
updates = [3, 10]

for batch in batchs:
  for update in updates:
    agents.append(DQN_Agent_PyTorch(get_params(batch, update)))


Using CPU
Using CPU
Using CPU
Using CPU


In [12]:
handler = Agent_handler({
    "num_episodes":1000,
    "max_steps":1000,
    "notify_percent":5,
    "skip": 15,
    "checkpoint_interval": 200,
    "crop": {
      "top": 33,
      "bottom": -16,
      "left": 0,
      "right": -1,
  }
})

In [13]:
results = handler.train([agents[0]], env)

~~~ Training Agent: 1/1 ~~~
	Name	: 8_3
	Start	: 11-30 04:43

	Episode	: 0/1000 0%

	!! CRASH !!


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x2304 and 1920x512)

In [ ]:
window_size = 5

for agent, result in results.items():
  moving_average = np.convolve(result["rewards"], np.ones(window_size)/window_size, mode='valid')

  padding = np.full(5 - 1, 0)
  result_array = np.concatenate([padding, moving_average])
  result["rewards averages"] = result_array
  Utils.plot_results(result, agent)

NameError: name 'results' is not defined